In [2]:
import pandas as pd

In [2]:
df = pd.read_pickle('merged_training.pkl')
df

,text,emotions
27383,i feel awful about it too because it s my job ...,sadness
110083,im alone i feel awful,sadness
140764,ive probably mentioned this before but i reall...,joy
100071,i was feeling a little low few days back,sadness
2837,i beleive that i am much more sensitive to oth...,love
...,...,...
566,that was what i felt when i was finally accept...,joy
36236,i take every day as it comes i m just focussin...,fear
76229,i just suddenly feel that everything was fake,sadness
131640,im feeling more eager than ever to claw back w...,joy


In [3]:
import requests
from datetime import datetime, timedelta

In [4]:
def get_headers(bearer_token):
    """Create headers for API requests"""
    return {
        "Authorization": f"Bearer {bearer_token}",
        "User-Agent": "v2UserTweetsLookup-Python"
    }

In [5]:
def get_user_id(bearer_token, username):
    """
    Get user ID from username

    Args:
        bearer_token (str): Twitter API bearer token
        username (str): Twitter username (without @)

    Returns:
        str: User ID if found, None otherwise
    """
    base_url = "https://api.x.com/2"
    url = f"{base_url}/users/by/username/{username}"

    try:
        response = requests.get(url, headers=get_headers(bearer_token))
        response.raise_for_status()

        return response.json()['data']['id']
    except requests.exceptions.RequestException as e:
        print(f"Error getting user ID: {e}")
        return None

In [7]:
def get_weekly_tweets(bearer_token, user_id):
    """
    Get tweets from the past week for a given user ID

    Args:
        bearer_token (str): Twitter API bearer token
        user_id (str): Twitter user ID

    Returns:
        list: List of tweets with their data
    """
    base_url = "https://api.twitter.com/2"

    # Calculate date range for past week
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(days=7)

    # Format dates as required by Twitter API
    start_time_str = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_time_str = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")

    url = f"{base_url}/users/{user_id}/tweets"

    # Define query parameters
    params = {
        "max_results": 100,  # Maximum allowed per request
        "start_time": start_time_str,
        "end_time": end_time_str,
        "tweet.fields": "created_at,text",  # Add more fields as needed
        "exclude": "retweets,replies"  # Only get original tweets
    }

    try:
        response = requests.get(url, headers=get_headers(bearer_token), params=params)
        response.raise_for_status()

        data = response.json()
        return data.get('data', [])
    except requests.exceptions.RequestException as e:
        print(f"Error getting tweets: {e}")
        return []


    {
    shraddhhaa13:1003304113017409537
    }
    

In [8]:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAALBlxAEAAAAAh6n7u79IAJTu7ia3CZDM9nJ0A0k%3DNH3X1zTtUi06JfBGsgjkk99doNMaFzd9VfdmNBaxwVa57otoXX'

    # Initial DataFrame to store cumulative tweets
cumulative_tweets_df = pd.DataFrame(columns=['user_id', 'username', 'created_at', 'text'])

    # List of usernames to collect tweets from
usernames = ["shraddhhaa13", "bandarmoment", "fr4nkoceansbf","chalkeat3r"]

for username in usernames:
        # Get user ID
    user_id = get_user_id(bearer_token, username)
    print(f"User ID for {username}: {user_id}")

    if not user_id:
        print(f"Could not find user with username: {username}")
        continue

        # Get weekly tweets
    tweets = get_weekly_tweets(bearer_token, user_id)
    print(f"Found {len(tweets)} tweets from the past week for {username}")

        # Convert tweets to DataFrame
    tweet_df = pd.DataFrame(tweets)
    tweet_df['user_id'] = user_id
    tweet_df['username'] = username

        # Convert created_at to datetime
    tweet_df['created_at'] = pd.to_datetime(tweet_df['created_at'], utc=True)

        # Reorder and select columns
    tweet_df = tweet_df[['user_id', 'username', 'created_at', 'text']]

        # Append to cumulative DataFrame
    cumulative_tweets_df = pd.concat([cumulative_tweets_df, tweet_df], ignore_index=True)

        # Rate limit handling: Sleep for 15 minutes between user requests
    print("Waiting 15 minutes to respect rate limits...")
    time.sleep(905)  # 15 minutes = 900 seconds

    # Display cumulative DataFrame
print("\nCumulative Tweet DataFrame:")
print(cumulative_tweets_df)

    # Optional: Save to CSV
cumulative_tweets_df.to_csv("cumulative_tweets.csv", index=False)
print("\nTweets saved to cumulative_tweets.csv")

User ID for shraddhhaa13: 1003304113017409537
Found 20 tweets from the past week for shraddhhaa13
Waiting 15 minutes to respect rate limits...


<ipython-input-8-7ff1d2171492>:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cumulative_tweets_df = pd.concat([cumulative_tweets_df, tweet_df], ignore_index=True)


User ID for bandarmoment: 1358168471406747649
Found 13 tweets from the past week for bandarmoment
Waiting 15 minutes to respect rate limits...
User ID for fr4nkoceansbf: 1567927430551334912
Found 21 tweets from the past week for fr4nkoceansbf
Waiting 15 minutes to respect rate limits...
User ID for chalkeat3r: 1838827096052797440
Found 59 tweets from the past week for chalkeat3r
Waiting 15 minutes to respect rate limits...

Cumulative Tweet DataFrame:
                 user_id      username                created_at  \
0    1003304113017409537  shraddhhaa13 2024-11-21 10:16:20+00:00   
1    1003304113017409537  shraddhhaa13 2024-11-20 19:24:48+00:00   
2    1003304113017409537  shraddhhaa13 2024-11-20 19:24:26+00:00   
3    1003304113017409537  shraddhhaa13 2024-11-20 19:07:50+00:00   
4    1003304113017409537  shraddhhaa13 2024-11-20 19:06:26+00:00   
..                   ...           ...                       ...   
108  1838827096052797440    chalkeat3r 2024-11-15 08:09:59+00:00   


In [9]:
cumulative_tweets_df

,user_id,username,created_at,text
0,1003304113017409537,shraddhhaa13,2024-11-21 10:16:20+00:00,#colonisedincoloniserdominatedfields https://t...
1,1003304113017409537,shraddhhaa13,2024-11-20 19:24:48+00:00,I need a yap session so bad ILL EXPLODE
2,1003304113017409537,shraddhhaa13,2024-11-20 19:24:26+00:00,I wanna take a walk but I can't find my earpho...
3,1003304113017409537,shraddhhaa13,2024-11-20 19:07:50+00:00,Got happy for 1 second I hope you rot in hell ...
4,1003304113017409537,shraddhhaa13,2024-11-20 19:06:26+00:00,Throwing myself off the building https://t.co/...
...,...,...,...,...
108,1838827096052797440,chalkeat3r,2024-11-15 08:09:59+00:00,oh thats not.. https://t.co/fXOLZmudE3
109,1838827096052797440,chalkeat3r,2024-11-15 08:09:08+00:00,i love winter but winter hates me hella my ski...
110,1838827096052797440,chalkeat3r,2024-11-15 08:08:02+00:00,im such a loser but im the coolest loser in a ...
111,1838827096052797440,chalkeat3r,2024-11-15 08:07:30+00:00,"my daily routine: open instagram, check main a..."
